# The New Pandas 2.0 Against Polars and Data.table - the fastest DataFrame libraries ever written
![](images/img.png)

### Installation

In [10]:
import datatable as dt
import pandas as pd
import polars as pl

In [11]:
dt.__version__

'1.0.0'

In [7]:
pd.__version__

'2.0.0rc0'

In [8]:
pl.__version__

'0.16.14'